## Assignment No.15-Q2
### Use Random Forest to prepare a model on fraud data 
### treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd
import numpy as np

from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split



In [46]:
df=pd.read_csv("/content/drive/MyDrive/Datasets/Random Forests/Fraud_check.csv")

In [47]:
df.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [49]:
df.isnull().sum()

Undergrad          0
Marital.Status     0
Taxable.Income     0
City.Population    0
Work.Experience    0
Urban              0
dtype: int64

#### No Null Values

In [50]:
#convert categorical data into numeric
Encoder=LabelEncoder()

In [51]:
df['Undergrad']=Encoder.fit_transform(df["Undergrad"])
df['Marital.Status']=Encoder.fit_transform(df["Marital.Status"])
df['Urban']=Encoder.fit_transform(df["Urban"])

In [52]:
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,0,2,68833,50047,10,1
1,1,0,33700,134075,18,1
2,0,1,36925,160205,30,1
3,1,2,50190,193264,15,1
4,0,1,81002,27533,28,0
...,...,...,...,...,...,...
595,1,0,76340,39492,7,1
596,1,0,69967,55369,2,1
597,0,0,47334,154058,0,1
598,1,1,98592,180083,17,0


In [53]:
df['income']=np.where(df['Taxable.Income']<=30000,'Risky',"Good")
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income
0,0,2,68833,50047,10,1,Good
1,1,0,33700,134075,18,1,Good
2,0,1,36925,160205,30,1,Good
3,1,2,50190,193264,15,1,Good
4,0,1,81002,27533,28,0,Good
...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,Good
596,1,0,69967,55369,2,1,Good
597,0,0,47334,154058,0,1,Good
598,1,1,98592,180083,17,0,Good


In [54]:
df1=df.drop(columns=['Taxable.Income'],axis=1)

In [55]:
df1

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,income
0,0,2,50047,10,1,Good
1,1,0,134075,18,1,Good
2,0,1,160205,30,1,Good
3,1,2,193264,15,1,Good
4,0,1,27533,28,0,Good
...,...,...,...,...,...,...
595,1,0,39492,7,1,Good
596,1,0,55369,2,1,Good
597,0,0,154058,0,1,Good
598,1,1,180083,17,0,Good


In [56]:
df1['income']=Encoder.fit_transform(df1["income"])

In [57]:
df1.head(2)

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,income
0,0,2,50047,10,1,0
1,1,0,134075,18,1,0


**0=GOOD**
**1=Risky**

In [58]:
#dividing data into x & y variable 
x=df1.iloc[:,:5]
y=df1['income']

In [59]:
#spliting data into training and testing

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=23)

### **Building Decision Tree Classifier using Entropy Criteria[C5.0]**

In [60]:
model=DecisionTreeClassifier(criterion="entropy",max_depth=3)
model.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [61]:
# predicting on test data set 
preds = model.predict(x_test) 

# getting the count of each category 
pd.Series(preds).value_counts() 

0    177
1      3
dtype: int64

In [62]:
# Accuracy 
np.mean(preds==y_test)*100

81.11111111111111

**Bagging**

In [63]:
# Bagged Decision Trees for Classification

kfold=KFold(n_splits=10)
cart=DecisionTreeClassifier()
num_tree=110

In [64]:
Bagging_model= BaggingClassifier(base_estimator=cart,n_estimators=num_tree)

In [65]:
Bagging_model.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=110)

In [66]:
result=cross_val_score(Bagging_model,x_train,y_train,cv=kfold)
print(result.mean())

0.7285714285714286


In [67]:
Bagging_preds = Bagging_model.predict(x_test)
pd.crosstab(y_test,Bagging_preds)

col_0,0,1
income,,
0,131,16
1,30,3


In [68]:
# Accuracy 
np.mean(Bagging_preds==y_test)*100

74.44444444444444

## **Random forest classification**

In [69]:
num_trees = 60
max_features = 3

kfold=KFold(n_splits=10, random_state=4,shuffle=True)
modelr=RandomForestClassifier(n_estimators=num_trees,max_features=max_features)
modelr.fit(x_train,y_train)


RandomForestClassifier(max_features=3, n_estimators=60)

In [70]:
result=cross_val_score(modelr,x_train,y_train,cv=kfold)
print(result.mean())

0.7404761904761905


In [71]:
# predicting on test data set 
preds = modelr.predict(x_test) 

pd.crosstab(preds,y_test)

income,0,1
row_0,,
0,134,30
1,13,3


In [72]:
# Accuracy 
np.mean(preds==y_test)*100

76.11111111111111

## **AdaBoost Classification**

In [73]:
num_tree=20
seed=7
kfold=KFold(n_splits=10)
modelA=AdaBoostClassifier(n_estimators=num_tree,random_state=seed)


modelA.fit(x_train,y_train)

result=cross_val_score(modelA,x_train,y_train,cv=kfold)
print(result.mean())

0.7642857142857142


In [74]:
# predicting on test data set 
preds = modelA.predict(x_test) 

pd.crosstab(preds,y_test)

income,0,1
row_0,,
0,146,33
1,1,0


In [75]:
# Accuracy 
np.mean(preds==y_test)*100

81.11111111111111